In [1]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import tensorflow
from keras import layers
from keras import models
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Fonctions de base

In [2]:
def loadImage(src):
    img=cv2.imread(src,0)
    return img

In [3]:
def printImage(I):
    # Affichage avec matplotlib (conversion BGR2RGB puis show)
    rgb = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)
    plt.imshow(rgb,interpolation='nearest')
    plt.show()

# Réduction du bruit

In [4]:
def reduceNoise(I, k):  # appliquer k = 7
    return cv2.medianBlur(I, k)

# Version A : Seuillage

In [5]:
def thresholding(I,s):  # appliquer s = 50
    ret,thresh1 = cv2.threshold(I,s,255,cv2.THRESH_BINARY)
    return thresh1

# Version B : Détection de contours de Canny

In [6]:
def edgesCanny(I, Vmin, Vmax):  # appliquer Vmin = 20 et Vmax = 150
    return cv2.Canny(I, Vmin, Vmax)

# Centrer sur la main

In [7]:
def cornersDetection(I):
    # Détection du point de contour le plus à gauche
    gauche = 0  # indice de la colonne du point de contour le plus à gauche
    j = 0
    while len(I[:,j][I[:,j]==255])==0 and j<len(I[0])-1:  # tant que les colonnes ne contiennent aucun pixel blanc
        j += 1 # parcours des colonnes de gauche à droite
    if j!=len(I[0])-1:
        gauche = j
        
    # Détection du point de contour le plus à droite
    droite = len(I[0])-1  # indice de la colonne du point de contour le plus à droite
    j = len(I[0])-1
    while len(I[:,j][I[:,j]==255])==0 and j>0:  # tant que les colonnes ne contiennent aucun pixel blanc
        j -= 1 # parcours des colonnes de droite à gauche
    if j!=0:
        droite = j
    
    return gauche, droite

In [8]:
def resizeImage(I):
    gauche, droite = cornersDetection(I)
    x = droite - gauche + 1
    if x < 240:
        if gauche < int((240-x)/2): # recadrage tout à gauche
            gauche = 0
            droite = 239
        elif len(I[0]) - droite < int((240-x)/2):  # recadrage tout à droite
            gauche = len(I[0])-239
            droite = len(I[0])
        else: # recadrage centré sur la main
            droite = droite + math.ceil((240-x)/2)
            gauche = gauche - int((240-x)/2)
    elif x > 240:  # troncature de l'image centrée sur la main
        droite = droite - math.ceil((x-240)/2)
        gauche = gauche + int((x-240)/2)
    return I[:,gauche:droite+1]

# Deep learning

In [9]:
dict_labels = {'01_palm': 0,
 '02_l': 1,
 '03_fist': 2,
 '04_fist_moved': 3,
 '05_thumb': 4,
 '06_index': 5,
 '07_ok': 6,
 '08_palm_moved': 7,
 '09_c': 8,
 '10_down': 9}

In [10]:
def upload_data(version): #version in ['raw', 'seuil', 'contours']
    X = []
    Y = []
    for i in range(0, 10): # Loop over the ten top-level folders
        for j in dict_labels.keys():
            for k in os.listdir('data/0' + str(i) + '/' + j + '/'):
                if not k.startswith('.'): # Again avoid hidden folders
                    img = loadImage('data/0' + str(i) + '/' + j + '/' + k)
                    if version == 'seuil':
                        img_seuil = thresholding(img,50)
                        X.append(resizeImage(img_seuil))
                    elif version == 'contours':
                        img_contours = edgesCanny(img, 20, 150)
                        X.append(resizeImage(img_contours))
                    else:
                        X.append(img)
                    Y.append(dict_labels[j])
    return X,Y

In [11]:
def permutation(X,Y):
    # Pour mélanger les images
    p = np.random.permutation(len(X))
    X_copy = list(X)
    Y_copy = list(Y)
    for i in range(len(X)):
        X[i] = X_copy[p[i]]
        Y[i] = Y_copy[p[i]]
    return(X,Y)

In [12]:
def scale_data(X,Y):
    # On normalise les données
    X = np.array(X)
    X = X.reshape((len(X), X[0].shape[0], X[0].shape[1], 1))
    X = X.astype('float32')/255
    Y = np.array(Y)
    Y = to_categorical(Y)
    return(X,Y)

In [13]:
def preprocessing_data(version):
    print('Uploading...')
    X,Y = upload_data(version)
    print('Permutation...')
    x,Y = permutation(X,Y)
    print('Scaling...')
    X,Y = scale_data(X,Y)
    print('Spliting...')
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2)
    return X_train, X_test, Y_train, Y_test

In [14]:
def create_CNN(optimizer, loss, metrics, input_shape):

    # On crée le modèle
    model = models.Sequential()

    # On ajoute des couches au modèle
    model.add(layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), padding='valid',  activation='relu', input_shape=input_shape)) 
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    
    # On compile
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

In [15]:
def create_MLP(optimizer, loss, metrics, input_dim):

    # On crée le modèle
    model_MLP = models.Sequential()

    # On ajoute des couches au modèle
    model_MLP.add(layers.core.Dense(64, input_shape=(input_dim,)))
    model_MLP.add(layers.core.Activation('relu'))
    model_MLP.add(layers.core.Dense(64))
    model_MLP.add(layers.core.Activation('relu'))
    model_MLP.add(layers.core.Dense(10))
    model_MLP.add(layers.core.Activation('softmax'))

    model_MLP.compile(optimizer=optimizer,loss=loss, metrics=metrics)
    
    return model_MLP

In [16]:
def fit_model(model, X_train, Y_train, X_test, Y_test, epochs=3, batch_size=64):
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size)
    score = model.evaluate(X_test, Y_test, batch_size=batch_size)
    return score

In [17]:
def compute_score(version):
    print('Preprocessing data...')
    X_train, X_test, Y_train, Y_test = preprocessing_data(version)
    
    # CNN
    print('Creating model CNN...')
    model_CNN = create_CNN('sgd', 'categorical_crossentropy', ['accuracy'], (X_train.shape[1], X_train.shape[2], X_train.shape[3]))
    print('CNN : ')
    score_cnn = fit_model(model_CNN, X_train, Y_train, X_test, Y_test)
    print('Loss : ', score_cnn[0])
    print('Accuracy : ', score_cnn[1])
    
    # MLP
    print('Creating model MLP...')
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3])
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2]*X_test.shape[3])
    model_MLP = create_MLP('sgd', 'categorical_crossentropy', ['accuracy'], X_train.shape[1])
    print('MLP : ')
    score_mlp = fit_model(model_MLP, X_train, Y_train, X_test, Y_test)
    print('Loss : ', score_mlp[0])
    print('Accuracy : ', score_mlp[1])

In [18]:
#print('RAW DATA')
#compute_score('raw')

print('SEUILLAGE')
compute_score('seuil')

print('CONTOURS')
compute_score('contours')

SEUILLAGE
Preprocessing data...
Uploading...
Permutation...
Scaling...
Spliting...
Creating model CNN...
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
CNN : 
Epoch 1/3
16000/16000 [==============================] - 2969s - loss: 1.1509 - acc: 0.6322   
Epoch 2/3
16000/16000 [==============================] - 2689s - loss: 0.2266 - acc: 0.9423   
Epoch 3/3
4000/4000 [==============================] - 223s   
Loss :  0.12429726666212082
Accuracy :  0.974
Creating model MLP...
MLP : 
Epoch 1/3
16000/16000 [==============================] - 25s - loss: 0.7709 - acc: 0.7849    
Epoch 2/3
16000/16000 [==============================] - 22s - loss: 0.1513 - acc: 0.9763    
Epoch 3/3
4000/4000 [==============================] - 2s     
Loss :  0.06633645966649056
Accuracy :  0.9885
CONTOURS
Preprocessing data...
Uploading...